In [16]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
import datetime
import xlrd
import openpyxl
import configparser
import os

In [17]:
class rschedule:
    'Class for Revit schedules'
    def __init__(self, proj, path_building):
        self.proj= proj
        self.path_building= path_building 
    def df (self):
        proj = self.proj
        path_building = self.path_building
        config = configparser.ConfigParser()
        config.read('configurations.ini')
        if (proj == 'TLC'):
            path_proj = config['path']['tlc']
        if (proj == 'ORC'): 
            path_proj = config['path']['orc']       
        list_folder = os.listdir(path_proj + r'/'+ path_building)    
        f = filter(lambda x: 'xlsx' in x, list_folder)
        list_folder = list(f)
        for i in range (len(list_folder)):
            if (r'~$' in list_folder[i]):
                list_folder.pop(i)
        df_folder = pd.Series(list_folder)
        df_folder = df_folder.str.split('.xlsx', expand=True)
        df_folder = df_folder.drop(1, axis = 1)
        df_folder
        df_folder_codes = df_folder[0].str.split('_', expand=True)
        df_folder_codes.rename(columns= {0: 'project', 
        1: 'stage',
        2: 'phase',
        3: 'building',
        4: 'company',
        5: 'type_file',
        6: 'discipline',
        7: 'discipline_rus',
        8: 'description'},
        inplace=True)
        xls_list = list(list_folder)
        listsht = list(list_folder)
        listdf = list(list_folder)
        df_mc = list(list_folder)
        df_mcmt = list(list_folder)
        df_m = list(list_folder)
        df_room = list(list_folder)
        for i in range(len(list_folder)):
            xls_list[i] = openpyxl.load_workbook(path_proj + r'/' + path_building + r'/' + list_folder[i])
            listsht[i] = xls_list[i].sheetnames
            listdf[i] = list(listsht[i])
        #Заполнение списка датафремов 
            for j in range(len(listsht[i])): 
                listdf[i][j] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = listsht[i][j])
                if (listdf[i][j].size == 0):
                    listdf[i][j] = pd.DataFrame(columns=['Family and Type', 'Count'])  
            for j in range(len(listsht[i])): 
                listdf[i][j] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = listsht[i][j])
                if (listdf[i][j].size == 0):
                    listdf[i][j] = pd.DataFrame(columns=['Family and Type', 'Count']) 
        #Заполнение пустых спецификаций
            for j in range(len(listsht[i])): 
                listdf[i][j] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = listsht[i][j])
                if (listdf[i][j].size == 0 or 'Department' in listdf[i][j].columns):
                    listdf[i][j] = pd.DataFrame(columns=['Family and Type', 'Count']) 
        #Установка мульти категорий           
            df_mc[i] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = 'VTBR_Multi-Category')
            df_mcmt[i] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = 'VTBR_Multi-Category MT')
        #Объединение мульти и мульти материал
            df_m[i] = pd.concat([df_mcmt[i],df_mc[i]], join = "outer")  
        #Установка датафр помещений  
            if 'VTBR_Room' in listsht[i]:
                df_room[i] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = 'VTBR_Room') 
        #Удаление мульти спец 
            for j in range(len(listsht[i])):         
                if ('Category' in listdf[i][j].columns):
                    listdf[i].append(listdf[i][j])
                    del listdf[i][j]
            for j in range(len(listsht[i])):           
                if ('Category' in listdf[i][j].columns):
                    listdf[i].append(listdf[i][j])
                    del listdf[i][j]
            del listdf[i][-2:]    
        #Добавление Category 
        for i in range(len(listdf)):
            for j in range(len(listdf[i])):          
                    listdf[i][j].insert(0, 'Building', df_folder_codes.building[i])
                    listdf[i][j].insert(1, 'Discipline', df_folder_codes.discipline[i])
                    listdf[i][j].insert(2, 'Category', (listdf[i][j].merge(df_m[i], on = 'Family and Type', how = 'left').Category))
                    listdf[i][j].insert(3, 'File', list_folder[i])            
        #Объединение спецификаций 
        df_cnc = list(listdf)
        for i in range(len(listdf)):
            df_cnc[i] = pd.concat([listdf[i][0]
                                   ], join = "outer")
            for j in range(1, len(listdf[i])):
                df_cnc[i] = pd.concat([df_cnc[i],
                                    listdf[i][j]
                                   ], join = "outer")

        df_cnc_building = pd.concat([df_cnc[0]
                                   ], join = "outer")
        for i in range(1, len(df_cnc)):
            df_cnc_building = pd.concat([df_cnc_building,
                                    df_cnc[i]
                                   ], join = "outer")

        return df_cnc_building 
    
    def files(self):
        proj = self.proj
        path_building = self.path_building
        config = configparser.ConfigParser()
        config.read('configurations.ini')
        if (proj == 'TLC'):
            path_proj = config['path']['tlc']
        if (proj == 'ORC'): 
            path_proj = config['path']['orc'] 
        list_folder = os.listdir(path_proj + r'/'+ path_building)    
        f = filter(lambda x: 'xlsx' in x, list_folder)
        list_folder = list(f)
        return list_folder 
 
    def path(self):
        proj = self.proj
        path_building = self.path_building
        config = configparser.ConfigParser()
        config.read('configurations.ini')
        if (proj == 'TLC'):
            path_proj = config['path']['tlc']
        if (proj == 'ORC'): 
            path_proj = config['path']['orc'] 
        return path_proj + '/' + path_building

In [18]:
# Examples 
bilding_orc_8 = rschedule('ORC', '8_Адм.блок_1 этап')
print('Путь к папке:', bilding_orc_8.path())
print('Список файлов:', bilding_orc_8.files())
#print('Собранная спецификация:', bilding_orc_8.df())


Путь к папке: \\192.168.128.12\VTBN_Disk\% ДПС\02 - ОРЦ\05 - ПИР\03 - ПРОЕКТ\04 - ТД\08 - BIM/8_Адм.блок_1 этап
Список файлов: ['ORC_T_I_8_STH_BIM_KR_Конструкции(ВГЗП)_Schedules.xlsx']


In [19]:
#Проверка на одном здании
# Read config
config = configparser.ConfigParser()
config.read('configurations.ini')
config.sections()
path_proj = config['path']['orc']
#path_proj = config['path']['tlc']
path_building = '1.1_МО Склад_3 этап'

list_folder = os.listdir(path_proj + r'/' + path_building)
f = filter(lambda x: 'xlsx' in x, list_folder)
list_folder = list(f)
list_folder
for i in range (len(list_folder)):
    if (r'~$' in list_folder[i]):
        list_folder.pop(i)
df_folder = pd.Series(list_folder)
df_folder = df_folder.str.split('.xlsx', expand=True)
df_folder = df_folder.drop(1, axis = 1)
df_folder
df_folder_codes = df_folder[0].str.split('_', expand=True)
df_folder_codes.rename(columns= {0: 'project', 
1: 'stage',
2: 'phase',
3: 'building',
4: 'company',
5: 'type_file',
6: 'discipline',
7: 'discipline_rus',
8: 'description'},
inplace=True)

df_folder_codes
path_building = '1.1_МО Склад_3 этап'
xls_list = list(list_folder)
listsht = list(list_folder)
listdf = list(list_folder)
df_mc = list(list_folder)
df_mcmt = list(list_folder)
df_m = list(list_folder)
df_room = list(list_folder)
for i in range(len(list_folder)):
    xls_list[i] = openpyxl.load_workbook(path_proj + r'/' + path_building + r'/' + list_folder[i])
    listsht[i] = xls_list[i].sheetnames
    listdf[i] = list(listsht[i])
#Заполнение списка датафремов 
    for j in range(len(listsht[i])): 
        listdf[i][j] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = listsht[i][j])
        if (listdf[i][j].size == 0):
            listdf[i][j] = pd.DataFrame(columns=['Family and Type', 'Count'])  
    for j in range(len(listsht[i])): 
        listdf[i][j] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = listsht[i][j])
        if (listdf[i][j].size == 0):
            listdf[i][j] = pd.DataFrame(columns=['Family and Type', 'Count']) 
#Заполнение пустых спецификаций
    for j in range(len(listsht[i])): 
        listdf[i][j] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = listsht[i][j])
        if (listdf[i][j].size == 0 or 'Department' in listdf[i][j].columns):
            listdf[i][j] = pd.DataFrame(columns=['Family and Type', 'Count']) 
#Установка мульти категорий           
    df_mc[i] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = 'VTBR_Multi-Category')
    df_mcmt[i] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = 'VTBR_Multi-Category MT')
#Объединение мульти и мульти материал
    df_m[i] = pd.concat([df_mcmt[i],df_mc[i]], join = "outer")  
#Установка датафр помещений  
    if 'VTBR_Room' in listsht[i]:
        df_room[i] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = 'VTBR_Room') 
#Удаление мульти спец 
    for j in range(len(listsht[i])):         
        if ('Category' in listdf[i][j].columns):
            listdf[i].append(listdf[i][j])
            del listdf[i][j]
    for j in range(len(listsht[i])):           
        if ('Category' in listdf[i][j].columns):
            listdf[i].append(listdf[i][j])
            del listdf[i][j]
    del listdf[i][-2:]    
#Добавление Category 
for i in range(len(listdf)):
    for j in range(len(listdf[i])):          
            listdf[i][j].insert(0, 'Building', df_folder_codes.building[i])
            listdf[i][j].insert(1, 'Discipline', df_folder_codes.discipline[i])
            listdf[i][j].insert(2, 'Category', (listdf[i][j].merge(df_m[i], on = 'Family and Type', how = 'left').Category))
            listdf[i][j].insert(3, 'File', list_folder[i])            
#Объединение спецификаций 
df_cnc = list(listdf)
for i in range(len(listdf)):
    df_cnc[i] = pd.concat([listdf[i][0]
                           ], join = "outer")
    for j in range(1, len(listdf[i])):
        df_cnc[i] = pd.concat([df_cnc[i],
                            listdf[i][j]
                           ], join = "outer")

df_cnc_building = pd.concat([df_cnc[0]
                           ], join = "outer")
for i in range(1, len(df_cnc)):
    df_cnc_building = pd.concat([df_cnc_building,
                            df_cnc[i]
                           ], join = "outer")

In [26]:
# Проверка Family and Type
list_true = list(listdf)
for i in range(len(listdf)):
#    print(i)
    for j in range(len(listdf[i])):
        list_true[i].append
#        print('Family and Type' in listdf[i][j],'---', j)
